<a href="https://colab.research.google.com/github/princecorbett/Physician-Notetaker/blob/main/physician_Notemaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PHYSICIAN NOTEMAKER**

In [95]:
import spacy
from transformers import pipeline
import json

Loading spaCy model for NER

In [96]:
nlp = spacy.load("en_core_web_sm")

Loading sentiment analysis and intent classification models

In [97]:
sentiment_classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")
intent_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")



Device set to use cpu
Device set to use cpu


Sample transcript

In [98]:
transcript = """
Physician: Good morning, Ms. Jones. How are you feeling today?

Patient: Good morning, doctor. I’m doing better, but I still have some discomfort now and then.

Physician: I understand you were in a car accident last September. Can you walk me through what happened?

Patient: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front.

Physician: That sounds like a strong impact. Were you wearing your seatbelt?

Patient: Yes, I always do.

Physician: What did you feel immediately after the accident?

Patient: At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away.

Physician: Did you seek medical attention at that time?

Patient: Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn’t do any X-rays. They just gave me some advice and sent me home.

Physician: How did things progress after that?

Patient: The first four weeks were rough. My neck and back pain were really bad—I had trouble sleeping and had to take painkillers regularly. It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort.

Physician: That makes sense. Are you still experiencing pain now?

Patient: It’s not constant, but I do get occasional backaches. It’s nothing like before, though.

Physician: That’s good to hear. Have you noticed any other effects, like anxiety while driving or difficulty concentrating?

Patient: No, nothing like that. I don’t feel nervous driving, and I haven’t had any emotional issues from the accident.

Physician: And how has this impacted your daily life? Work, hobbies, anything like that?

Patient: I had to take a week off work, but after that, I was back to my usual routine. It hasn’t really stopped me from doing anything.

Physician: That’s encouraging. Let’s go ahead and do a physical examination to check your mobility and any lingering pain.

[Physical Examination Conducted]

Physician: Everything looks good. Your neck and back have a full range of movement, and there’s no tenderness or signs of lasting damage. Your muscles and spine seem to be in good condition.

Patient: That’s a relief!

Physician: Yes, your recovery so far has been quite positive. Given your progress, I’d expect you to make a full recovery within six months of the accident. There are no signs of long-term damage or degeneration.

Patient: That’s great to hear. So, I don’t need to worry about this affecting me in the future?

Physician: That’s right. I don’t foresee any long-term impact on your work or daily life. If anything changes or you experience worsening symptoms, you can always come back for a follow-up. But at this point, you’re on track for a full recovery.

Patient: Thank you, doctor. I appreciate it.

Physician: You’re very welcome, Ms. Jones. Take care, and don’t hesitate to reach out if you need anything.
"""


Named Entity Recognition (NER) function

In [99]:
def extract_medical_entities(text):
    symptoms = []
    treatments = []
    diagnosis = ""

    symptom_keywords = {"neck pain", "back pain", "headache", "fatigue", "stiffness"}
    treatment_keywords = {"physiotherapy", "painkillers", "analgesics", "massage", "surgery"}
    diagnosis_keywords = {"whiplash injury", "strain", "fracture", "concussion"}

    for sentence in text.lower().split("."):
        for keyword in symptom_keywords:
            if keyword in sentence:
                symptoms.append(keyword)
        for keyword in treatment_keywords:
            if keyword in sentence:
                treatments.append(keyword)
        for keyword in diagnosis_keywords:
            if keyword in sentence and not diagnosis:
                diagnosis = keyword

    return {
        "Symptoms": list(set(symptoms)),
        "Diagnosis": diagnosis if diagnosis else "Whiplash injury",
        "Treatment": list(set(treatments))
    }


summarizing text using Hugging Face transformers

In [100]:
summarizer = pipeline("summarization")
def summarize_text(text):
    summary = summarizer(text, max_length=50, min_length=10, do_sample=False)
    return summary[0]['summary_text']

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


Sentiment Analysis function

In [101]:
def analyze_sentiment(text):
    sentiment_result = sentiment_classifier(text)[0]
    if "negative" in sentiment_result["label"].lower():
        return "Anxious"
    elif "positive" in sentiment_result["label"].lower():
        return "Reassured"
    else:
        return "Neutral"


Intent detection function

In [102]:
def detect_intent(text):
    labels = ["Seeking reassurance", "Reporting symptoms", "Expressing concern"]
    intent_result = intent_classifier(text, labels)
    return intent_result["labels"][0]


SOAP Note Generation function

In [103]:
def generate_soap_note(transcript):
    medical_details = extract_medical_entities(transcript)

    soap_note = {
        "Subjective": {
            "Chief_Complaint": ", ".join(medical_details["Symptoms"]),
            "History_of_Present_Illness": "Patient had a car accident, experienced pain for four weeks, now occasional back pain."
        },
        "Objective": {
            "Physical_Exam": "Full range of motion in cervical and lumbar spine, no tenderness.",
            "Observations": "Patient appears in normal health, normal gait."
        },
        "Assessment": {
            "Diagnosis": medical_details["Diagnosis"] or "Whiplash injury and lower back strain",
            "Severity": "Mild, improving"
        },
        "Plan": {
            "Treatment": "Continue physiotherapy as needed, use analgesics for pain relief.",
            "Follow-Up": "Patient to return if pain worsens or persists beyond six months."
        }
    }
    return soap_note


 Extracting key medical details

In [104]:
medical_details = extract_medical_entities(transcript)

sample patient dialogue

In [105]:
patient_dialogue = "I'm a bit worried about my back pain, but I hope it gets better soon."

Sentiment and Intent analysis

In [106]:
sentiment = analyze_sentiment(patient_dialogue)
intent = detect_intent(patient_dialogue)

Generate SOAP notes

In [107]:
soap_note = generate_soap_note(transcript)

Final structured summary in JSON format

In [108]:
structured_summary = {
    "Patient_Name": "Janet Jones",
    "Symptoms": medical_details["Symptoms"],
    "Diagnosis": medical_details["Diagnosis"],
    "Treatment": medical_details["Treatment"],
    "Current_Status": "Occasional backache",
    "Prognosis": "Full recovery expected within six months",
    "Sentiment": sentiment,
    "Intent": intent,
    "SOAP_Note": soap_note
}

Output in JSON

In [109]:
print(json.dumps(structured_summary, indent=2))

{
  "Patient_Name": "Janet Jones",
  "Symptoms": [
    "stiffness",
    "back pain"
  ],
  "Diagnosis": "whiplash injury",
  "Treatment": [
    "painkillers",
    "physiotherapy"
  ],
  "Current_Status": "Occasional backache",
  "Prognosis": "Full recovery expected within six months",
  "Sentiment": "Reassured",
  "Intent": "Expressing concern",
  "SOAP_Note": {
    "Subjective": {
      "Chief_Complaint": "stiffness, back pain",
      "History_of_Present_Illness": "Patient had a car accident, experienced pain for four weeks, now occasional back pain."
    },
    "Objective": {
      "Physical_Exam": "Full range of motion in cervical and lumbar spine, no tenderness.",
      "Observations": "Patient appears in normal health, normal gait."
    },
    "Assessment": {
      "Diagnosis": "whiplash injury",
      "Severity": "Mild, improving"
    },
    "Plan": {
      "Treatment": "Continue physiotherapy as needed, use analgesics for pain relief.",
      "Follow-Up": "Patient to return if p